In [18]:
import bs4 as bs
import datetime as dt
import os
import pandas as pd
import pandas_datareader.data as web
import pickle ##helps in serializing the object
import requests
import matplotlib.pyplot as plt
from matplotlib import style
import numpy as np
from collections import Counter

from sklearn import svm, cross_validation, neighbors
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

import json

In [19]:
def save_sp500_tickers():
    resp=requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup=bs.BeautifulSoup(resp.text,"lxml")
    
    
    table=soup.find('table',{'class':'wikitable sortable'})
    tickers=[]
    for row in table.findAll('tr')[1:]:
        ticker=row.findAll('td')[0].text
        tickers.append(ticker)
    
    with open("sp500tickers.pickle","wb") as f:
        pickle.dump(tickers,f)
    #print(tickers)
    return(tickers)

save_sp500_tickers()

['MMM',
 'ABT',
 'ABBV',
 'ACN',
 'ATVI',
 'AYI',
 'ADBE',
 'AMD',
 'AAP',
 'AES',
 'AET',
 'AMG',
 'AFL',
 'A',
 'APD',
 'AKAM',
 'ALK',
 'ALB',
 'ARE',
 'ALXN',
 'ALGN',
 'ALLE',
 'AGN',
 'ADS',
 'LNT',
 'ALL',
 'GOOGL',
 'GOOG',
 'MO',
 'AMZN',
 'AEE',
 'AAL',
 'AEP',
 'AXP',
 'AIG',
 'AMT',
 'AWK',
 'AMP',
 'ABC',
 'AME',
 'AMGN',
 'APH',
 'APC',
 'ADI',
 'ANDV',
 'ANSS',
 'ANTM',
 'AON',
 'AOS',
 'APA',
 'AIV',
 'AAPL',
 'AMAT',
 'ADM',
 'ARNC',
 'AJG',
 'AIZ',
 'T',
 'ADSK',
 'ADP',
 'AZO',
 'AVB',
 'AVY',
 'BHGE',
 'BLL',
 'BAC',
 'BK',
 'BCR',
 'BAX',
 'BBT',
 'BDX',
 'BRK.B',
 'BBY',
 'BIIB',
 'BLK',
 'HRB',
 'BA',
 'BWA',
 'BXP',
 'BSX',
 'BHF',
 'BMY',
 'AVGO',
 'BF.B',
 'CHRW',
 'CA',
 'COG',
 'CPB',
 'COF',
 'CAH',
 'CBOE',
 'KMX',
 'CCL',
 'CAT',
 'CBG',
 'CBS',
 'CELG',
 'CNC',
 'CNP',
 'CTL',
 'CERN',
 'CF',
 'SCHW',
 'CHTR',
 'CHK',
 'CVX',
 'CMG',
 'CB',
 'CHD',
 'CI',
 'XEC',
 'CINF',
 'CTAS',
 'CSCO',
 'C',
 'CFG',
 'CTXS',
 'CLX',
 'CME',
 'CMS',
 'COH',
 'KO',
 'C

In [20]:
def get_data(reload_sp500=False):
    if reload_sp500:
        tickers=save_sp500_tickers()
    else:
        with open("sp500tickers.pickle","rb") as f:
            tickers = pickle.load(f) #read bytes
            
    if not os.path.exists('stock_dfs'):
        os.makedirs('stock_dfs')
    
    start = dt.datetime(2006,1,1)
    end=dt.datetime(2016,12,31)
    
    for ticker in tickers:
        print(ticker)
        if not os.path.exists('stock_dfs/{}.csv'.format(ticker)):
            df= web.DataReader(ticker,'google',start, end)
            df.to_csv('stock_dfs/{}.csv'.format(ticker))
        else:
            print('Already have {}'.format(ticker))

In [5]:
get_data()

MMM
Already have MMM
ABT
Already have ABT
ABBV
Already have ABBV
ACN
Already have ACN
ATVI
Already have ATVI
AYI
Already have AYI
ADBE
Already have ADBE
AMD
Already have AMD
AAP
Already have AAP
AES
Already have AES
AET
Already have AET
AMG
Already have AMG
AFL
Already have AFL
A
Already have A
APD
Already have APD
AKAM
Already have AKAM
ALK
Already have ALK
ALB
Already have ALB
ARE
Already have ARE
ALXN
Already have ALXN
ALGN
Already have ALGN
ALLE
Already have ALLE
AGN
Already have AGN
ADS
Already have ADS
LNT
Already have LNT
ALL
Already have ALL
GOOGL
Already have GOOGL
GOOG
Already have GOOG
MO
Already have MO
AMZN
Already have AMZN
AEE
Already have AEE
AAL
Already have AAL
AEP
Already have AEP
AXP
Already have AXP
AIG
Already have AIG
AMT
Already have AMT
AWK
Already have AWK
AMP
Already have AMP
ABC
Already have ABC
AME
Already have AME
AMGN
Already have AMGN
APH
Already have APH
APC
Already have APC
ADI
Already have ADI
ANDV
Already have ANDV
ANSS
Already have ANSS
ANTM
Already

RemoteDataError: Unable to read URL: http://www.google.com/finance/historical?q=LMT&startdate=Jan+01%2C+2006&enddate=Dec+31%2C+2016&output=csv

In [21]:
tickers=['AMZN']
for ticker in tickers:   
    portfolioAnalysisRequest= requests.get("https://www.blackrock.com/tools/hackathon/performance", params= {'identifiers':ticker,"fromDate":20160101, "toDate":20161212,'outputDataExpression':"resultMap['RETURNS'][0]['returnsMap']" })
    portfolioAnalysisRequest.text # get in text string format
    portfolioAnalysisRequest.json # get as json object                                       
    print(portfolioAnalysisRequest.text)

{"20040401":{"asOfDate":20040401,"drawdown":0,"level":1,"oneDay":0,"sinceStartDate":0,"sinceStartDateAnnualized":0},"20040402":{"asOfDate":20040402,"drawdown":0,"level":1.030174,"oneDay":0.030174,"sinceStartDate":0.030174,"sinceStartDateAnnualized":51564.99923540529562160372734069824},"20040403":{"asOfDate":20040403,"drawdown":0,"level":1.030174,"oneDay":0,"sinceStartDate":0.030174,"sinceStartDateAnnualized":226.0814814893660411598830251023173},"20040404":{"asOfDate":20040404,"drawdown":0,"level":1.030174,"oneDay":0,"sinceStartDate":0.030174,"sinceStartDateAnnualized":36.22098077346301181478338548913598},"20040405":{"asOfDate":20040405,"drawdown":0,"level":1.049619564424,"oneDay":0.018876,"sinceStartDate":0.049619564424,"sinceStartDateAnnualized":82.01612171100690318326087435707450},"20040406":{"asOfDate":20040406,"drawdown":0,"level":1.037997126987133048,"oneDay":-0.011073,"sinceStartDate":0.037997126987133048,"sinceStartDateAnnualized":14.21665016682934101766022649826482},"20040407":

In [38]:
portfolioAnalysisRequest.text

'{"20040401":{"asOfDate":20040401,"drawdown":0,"level":1,"oneDay":0,"sinceStartDate":0,"sinceStartDateAnnualized":0},"20040402":{"asOfDate":20040402,"drawdown":0,"level":1.030174,"oneDay":0.030174,"sinceStartDate":0.030174,"sinceStartDateAnnualized":51564.99923540529562160372734069824},"20040403":{"asOfDate":20040403,"drawdown":0,"level":1.030174,"oneDay":0,"sinceStartDate":0.030174,"sinceStartDateAnnualized":226.0814814893660411598830251023173},"20040404":{"asOfDate":20040404,"drawdown":0,"level":1.030174,"oneDay":0,"sinceStartDate":0.030174,"sinceStartDateAnnualized":36.22098077346301181478338548913598},"20040405":{"asOfDate":20040405,"drawdown":0,"level":1.049619564424,"oneDay":0.018876,"sinceStartDate":0.049619564424,"sinceStartDateAnnualized":82.01612171100690318326087435707450},"20040406":{"asOfDate":20040406,"drawdown":0,"level":1.037997126987133048,"oneDay":-0.011073,"sinceStartDate":0.037997126987133048,"sinceStartDateAnnualized":14.21665016682934101766022649826482},"20040407"

In [ ]:
for key in portfolioAnalysisRequest.text:
    print(key)
print("\n")

In [ ]:
jsonRes = json.load(portfolioAnalysisRequest.text)

In [ ]:
print(jsonRes)

In [34]:
portfolioAnalysisRequest['20040401']

TypeError: 'Response' object is not subscriptable

In [31]:
print(portfolioAnalysisRequest['oneDay'])

TypeError: 'Response' object is not subscriptable